In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json

In [2]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
CIRCOM_PRIME = 21888242871839275222246405745257275088548364400416034343698204186575808495617
MAX_POSITIVE = CIRCOM_PRIME // 2
MAX_NEGATIVE = MAX_POSITIVE + 1 # The most positive number
CIRCOM_NEGATIVE_1 = 21888242871839275222246405745257275088548364400416034343698204186575808495617 - 1
EXPONENT = 15

class SeparableConv2D(nn.Module):
    '''Separable convolution'''
    def __init__(self, in_channels, out_channels, stride=1):
        super(SeparableConv2D, self).__init__()
        self.dw_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pw_conv =  nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)

    def forward(self, x):
        x = self.dw_conv(x)
        x = self.pw_conv(x)
        return x

def from_circom(x):
    if type(x) != int:
        x = int(x)
    if x > MAX_POSITIVE: 
        return x - CIRCOM_PRIME
    return x
    
def to_circom(x):
    if type(x) != int:
        x = int(x)
    if x < 0:
        return x + CIRCOM_PRIME 
    return x

In [3]:
# def DepthwiseConv(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
#     assert(nFilters % nChannels == 0)
#     outRows = (nRows - kernelSize)//strides + 1
#     outCols = (nCols - kernelSize)//strides + 1
    
#     out = np.zeros((outRows, outCols, nFilters))
#     remainder = np.zeros((outRows, outCols, nFilters))
    
#     for row in range(outRows):
#         for col in range(outCols):
#             for channel in range(nChannels):
#                 for x in range(kernelSize):
#                     for y in range(kernelSize):
#                         out[row, col, channel] += input[row*strides+x, col*strides+y, channel] * weights[x, y, channel]
                
#                 out[row][col][channel] += bias[channel]
#                 remainder[row][col][channel] = out[row][col][channel] % n
#                 out[row][col][channel] = out[row][col][channel] / n
                            
#     return out, remainder
    
# def PointwiseConv2d(nRows, nCols, nChannels, nFilters, strides, n, input, weights, bias):
#     kernelSize = 1
#     outRows = (nRows - kernelSize)//strides + 1
#     outCols = (nCols - kernelSize)//strides + 1
#     out = np.zeros((outRows, outCols, nFilters))
#     for row in range(outRows):
#         for col in range(outCols):
#             for filter in range(nFilters):
#                 for k in range(nChannels):
#                     out[row, col, filter] += input[row*strides, col*strides, k] * weights[k, filter]
                    
#                 out[row][col][filter] += bias[filter]
#                 out[row][col][filter] = out[row][col][filter] / n
                            
#     return out

# def SeparableConvImpl(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
#     outRows = (nRows - kernelSize)//strides + 1
#     outCols = (nCols - kernelSize)//strides + 1

#     depthOut, rem = DepthwiseConv(nRows, nCols, nChannels, nDepthFilters, kernelSize, strides, n, input, depthWeights, depthBias)
#     pointOut = PointwiseConv2d(outRows, outCols, nChannels, nPointFilters, strides, n, depthOut, pointWeights, pointBias)
#     return pointOut

In [4]:
# def DepthwiseConvInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
#     assert(nFilters % nChannels == 0)
#     outRows = (nRows - kernelSize)//strides + 1
#     outCols = (nCols - kernelSize)//strides + 1
    
#     Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
#     Weights = [[[str(weights[i][j][k] % p) for k in range(nChannels)] for j in range(kernelSize)] for i in range(kernelSize)]
#     Bias = [str(bias[i] % p) for i in range(nFilters)]
#     out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
#     str_out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
#     remainder = [[[None for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    
#     for row in range(outRows):
#         for col in range(outCols):
#             for channel in range(nChannels):
#                 for x in range(kernelSize):
#                     for y in range(kernelSize):
#                         out[row][col][channel] += int(input[row*strides+x][col*strides+y][channel]) * int(weights[x][y][channel])
                        
#                 out[row][col][channel] += int(bias[channel])
#                 remainder[row][col][channel] = str(int(out[row][col][channel] % n))
#                 out[row][col][channel] = int((out[row][col][channel] // n))
#                 str_out[row][col][channel] = str(out[row][col][channel] % p)
                
#     return Input, Weights, Bias, out, str_out, remainder
    
# def PointwiseConv2dInt(nRows, nCols, nChannels, nFilters, strides, n, input, weights, bias):
#     kernelSize = 1
#     outRows = (nRows - kernelSize)//strides + 1
#     outCols = (nCols - kernelSize)//strides + 1
    
#     Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
#     Weights = [[str(weights[k][l] % p) for l in range(nFilters)] for k in range(nChannels)]
#     Bias = [str(bias[i] % p) for i in range(nFilters)]
#     out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
#     str_out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
#     remainder = [[[None for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    
#     for row in range(outRows):
#         for col in range(outCols):
#             for filter in range(nFilters):
#                 for channel in range(nChannels):
#                     out[row][col][filter] += int(input[row*strides][col*strides][channel]) * int(weights[channel][filter])
                            
#                 out[row][col][filter] += int(bias[filter])
#                 remainder[row][col][filter] = str(int(out[row][col][filter] % n))
#                 out[row][col][filter] = str(out[row][col][filter] // n % p)
#     return Input, Weights, Bias, out, remainder

# def SeparableConvInt(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
#     outRows = (nRows - kernelSize)//strides + 1
#     outCols = (nCols - kernelSize)//strides + 1

#     Input, DepthWeights, DepthBias, depthOut, depthStrOut, depthRemainder = DepthwiseConvInt(nRows, nCols, nChannels, nDepthFilters, kernelSize, strides, n, input, depthWeights, depthBias)
#     test = [[[from_circom(int(depthStrOut[i][j][k])) for k in range(3)] for j in range(5)] for i in range(5)]
#     assert(test == depthOut)
#     pInput, PointWeights, PointBias, pointOut, pointRem = PointwiseConv2dInt(outRows, outCols, nChannels, nPointFilters, strides, n, depthOut, pointWeights, pointBias)
    
#     return Input, DepthWeights, DepthBias, depthOut, depthStrOut, depthRemainder, PointWeights, PointBias, pointOut, pointRem

In [1]:
def DepthwiseConv(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    assert(nFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    # out = np.zeros((outRows, outCols, nFilters))
    out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    remainder = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    # remainder = np.zeros((outRows, outCols, nFilters))
    
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        out[row][col][channel] += int(input[row*strides+x, col*strides+y, channel]) * int(weights[x, y, channel])
                
                out[row][col][channel] += int(bias[channel])
                remainder[row][col][channel] = str(int(out[row][col][channel] % n))
                out[row][col][channel] = int(out[row][col][channel] // n)
                            
    return out, remainder
    
def PointwiseConv2d(nRows, nCols, nChannels, nFilters, strides, n, input, weights, bias):
    kernelSize = 1
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    str_out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    remainder = [[[None for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    for row in range(outRows):
        for col in range(outCols):
            for filter in range(nFilters):
                for k in range(nChannels):
                    out[row][col][filter] += int(input[row*strides][col*strides][k]) * int(weights[k, filter])
                    
                out[row][col][filter] += int(bias[filter])
                remainder[row][col][filter] = str(int(out[row][col][filter] % n))
                out[row][col][filter] = int(out[row][col][filter] // n)
                str_out[row][col][filter] = str(out[row][col][filter] % p)
                            
    return out, str_out, remainder
    
def SeparableConvImpl(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1

    depth_out, depth_remainder = DepthwiseConv(nRows, nCols, nChannels, nDepthFilters, kernelSize, strides, n, input, depthWeights, depthBias)
    point_out, point_str_out, point_remainder = PointwiseConv2d(outRows, outCols, nChannels, nPointFilters, strides, n, depth_out, pointWeights, pointBias)
    return depth_out, depth_remainder, point_out, point_str_out, point_remainder

In [2]:
input = torch.randn((1, 3, 5, 5))
model = SeparableConv2D(3, 6)

NameError: name 'torch' is not defined

In [3]:
# weights = model.dw_conv.weight.squeeze().detach().numpy()
# bias = torch.zeros(weights.shape[0]).numpy()

# expected = model.dw_conv(input).detach().numpy()

# padded = F.pad(input, (1,1,1,1), "constant", 0) # Padding for convolution with "same" configuration
# padded = padded.squeeze().numpy().transpose((1, 2, 0))
# weights = weights.transpose((1, 2, 0))

# actual, rem = DepthwiseConv(7, 7, 3, 3, 3, 1, 1, padded, weights, bias)
# expected = expected.squeeze().transpose((1, 2, 0))

# assert(np.allclose(expected, actual, atol=0.00001))

In [8]:
# weights = model.pw_conv.weight.detach().numpy()
# print(f"{weights.shape=}")
# bias = torch.zeros(weights.shape[0]).numpy()

# expected = model.pw_conv(input).detach().numpy()

# padded = input.squeeze().numpy().transpose((1, 2, 0))
# print(padded.shape)
# weights = weights.transpose((2, 3, 1, 0)).squeeze()

# actual = PointwiseConv2d(5, 5, 3, 6, 1, 1, padded, weights, bias)

# expected = expected.squeeze().transpose((1, 2, 0))

# assert(np.allclose(expected, actual, atol=0.00001))

In [9]:
# depthWeights = model.dw_conv.weight.squeeze().detach().numpy()
# depthBias = torch.zeros(depthWeights.shape[0]).numpy()

# depthWeights = depthWeights.transpose((1, 2, 0))

# pointWeights = model.pw_conv.weight.detach().numpy()
# print(f"{depthWeights.shape=}")
# pointBias = torch.zeros(pointWeights.shape[0]).numpy()
# pointWeights = pointWeights.transpose((2, 3, 1, 0)).squeeze()

# expected = model(input).detach().numpy()
# print(f"{expected.shape=}")

# padded = F.pad(input, (1,1,1,1), "constant", 0) # Padding for convolution with "same" configuration
# padded = padded.squeeze().numpy().transpose((1, 2, 0))

# print(pointBias.shape)
# actual = SeparableConvImpl(7, 7, 3, 3, 6, 3, 1, 1, padded, depthWeights, pointWeights, depthBias, pointBias)
# expected = expected.squeeze().transpose((1, 2, 0))

# assert(np.allclose(expected, actual, atol=0.00001))

In [10]:
depthWeights = model.dw_conv.weight.squeeze().detach().numpy()
depthBias = torch.zeros(depthWeights.shape[0]).numpy()

depthWeights = depthWeights.transpose((1, 2, 0))

pointWeights = model.pw_conv.weight.detach().numpy()
print(f"{depthWeights.shape=}")
pointBias = torch.zeros(pointWeights.shape[0]).numpy()
pointWeights = pointWeights.transpose((2, 3, 1, 0)).squeeze()

expected = model(input).detach().numpy()
print(f"{expected.shape=}")

padded = F.pad(input, (1,1,1,1), "constant", 0) # Padding for convolution with "same" configuration
padded = padded.squeeze().numpy().transpose((1, 2, 0))

print(pointBias.shape)
# actual = SeparableConvImpl(7, 7, 3, 3, 6, 3, 1, 1, padded, depthWeights, pointWeights, depthBias, pointBias)
quantized_image = padded * 10**EXPONENT
quantized_depth_weights = depthWeights * 10**EXPONENT
quantized_point_weights = pointWeights * 10**EXPONENT

depth_out, depth_remainder, point_out, point_str_out, point_remainder = SeparableConvImpl(7, 7, 3, 3, 6, 3, 1, 10**EXPONENT, quantized_image.round().astype(int), quantized_depth_weights.round().astype(int), quantized_point_weights.round().astype(int), depthBias.astype(int), pointBias.astype(int))

actual_scaled = [[[point_out[i][j][k] / 10**EXPONENT for k in range(6)] for j in range(5)] for i in range(5)]

expected = expected.squeeze().transpose((1, 2, 0))

assert(np.allclose(expected, actual_scaled, atol=0.00001))


# Input, DepthWeights, DepthBias, depthOut, depthStrOut, DepthRem, PointWeights, PointBias, pointOut, pointRem = SeparableConvInt(7, 7, 3, 3, 6, 3, 1, 10**EXPONENT, quantized_image.round().astype(int), quantized_depth_weights.round().astype(int), quantized_point_weights.round().astype(int), depthBias.astype(int), pointBias.astype(int))

circuit_in = quantized_image.round().astype(int).astype(str).tolist()
circuit_depth_weights = quantized_depth_weights.round().astype(int).astype(str).tolist()
circuit_point_weights = quantized_point_weights.round().astype(int).astype(str).tolist()
circuit_depth_bias = depthBias.round().astype(int).astype(str).tolist()
circuit_point_bias = pointBias.round().astype(int).astype(str).tolist()


input_json_path = "separableConv2D_input.json"
with open(input_json_path, "w") as input_file:
    json.dump({"in": circuit_in,
               "depthWeights": circuit_depth_weights,
               "depthBias": circuit_depth_bias,
               "depthRemainder": depth_remainder,
               "depthOut": depth_out,
               
               "pointWeights": circuit_point_weights,
               "pointBias": circuit_point_bias,
               "pointRemainder": point_remainder,
               "pointOut": point_str_out,
              },
              input_file)


depthWeights.shape=(3, 3, 3)
expected.shape=(1, 6, 5, 5)
(6,)


NameError: name 'depthOut' is not defined

In [44]:
test = np.array(actual)
test[0][0]

array([ 289246014266912,  458852178050435, -104551710192411,
        -85286796832706,   70991076566637, -373719950995314])

In [45]:
expected[0][0]

array([ 0.289246  ,  0.45885214, -0.10455171, -0.0852868 ,  0.07099106,
       -0.37371993], dtype=float32)